In [ ]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from mpl_toolkits.basemap import Basemap
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import cPickle
import os
os.chdir('/Users/zbutler/research/fire_prediction')
from data import data
import prediction.cluster_regression
reload(prediction.cluster_regression)
from prediction.cluster_regression import ClusterRegression
from prediction.poisson_regression import evaluate_glm
from plotting.plot_map import draw_map_nogrid_static
from carpentry.get_gfs_and_modis_v2 import gfs_to_loc_df
from geometry.grid_conversion import ak_bb
import matplotlib
%matplotlib inline

In [ ]:
clust_feat_df = data.load_clust_feat_df(clust_thresh=5)
ct = 711
clust = clust_df[clust_df.cluster == ct]

In [ ]:
# Figure 1a: Alaska with a heatmap of detections for 2013
# TODO: Make sure we don't have alignment issues
year = 2013
annual_fires = modis[modis.year==year]
lats = gfs['lats']
lat_res = (lats[0,0] - lats[1,0]) / 2.
lons = gfs['lons']
lon_res = (lons[0,1] - lons[0,0]) / 2.

fire_map = np.zeros(lats.shape)
for row in xrange(lats.shape[0]):
    lat = lats[row,0]
    for col in xrange(lats.shape[1]):
        lon = lons[row,col]
        neg_lon = lon - 360
        fire_map[row,col] = len(annual_fires[(annual_fires.lat >= (lat - lat_res)) & (annual_fires.lat < (lat + lat_res)) &
                                             (annual_fires.long >= (neg_lon - lat_res)) & (annual_fires.long < (neg_lon + lon_res))])

plt.figure(figsize=(10,10))
ak_bb = [lats[-1,0], lats[0,0], lons[0,0], lons[0,-1]]
mp = Basemap(projection="merc",
             lat_0=ak_bb[0], lon_0=ak_bb[2],
             llcrnrlat=ak_bb[0],
             llcrnrlon=ak_bb[2],
             urcrnrlat=ak_bb[1],
             urcrnrlon=ak_bb[3],
             resolution='i')
mp.drawcoastlines(color='w')
mx = np.max(fire_map)
mn = np.min(fire_map)
mp.imshow(fire_map, vmin=mn, vmax=mx)
mp.colorbar()
plt.title("Number of detections per grid cell in year %d" % year)
plt.savefig("pics/detections_heatmap_%d.png" % year)
plt.show()

In [ ]:
# Figure 1a: Alaska with a heatmap of detections for 2013
year = 2013
from geometry.grid_conversion import ak_bb
annual_fires = modis[modis.year==year]

matplotlib.rcParams.update({'font.size': 14})
plt.figure(figsize=(10,10))
ak_bb = [54, 71.5, -168, -138]
mp = Basemap(projection="merc",
             lat_0=ak_bb[0], lon_0=ak_bb[0],
             llcrnrlat=ak_bb[0],
             llcrnrlon=ak_bb[2],
             urcrnrlat=ak_bb[1],
             urcrnrlon=ak_bb[3],
             resolution='i')

mp.drawcoastlines()
#mp.etopo()
#mp.shadedrelief()
mp_lons, mp_lats = mp(np.array(annual_fires.long), np.array(annual_fires.lat))
mp.plot(mp_lons, mp_lats, 'r.', markersize=1)
ct_lons, ct_lats = mp(np.array(clust.long), np.array(clust.lat))
mp.plot(ct_lons, ct_lats, 'b.', markersize=1)
parallels = np.arange(ak_bb[0],ak_bb[1],2.5)
mp.drawparallels(parallels, labels=[True,False,False,False], linewidth=0.2)
meridians = np.arange(ak_bb[2],ak_bb[3],5.)
mp.drawmeridians(meridians, labels=[False,False,False,True], linewidth=0.2)
#plt.title("Number of detections per grid cell in year %d" % year)
plt.savefig("pics/detections_plot_%d.png" % year, bbox_inches='tight')
plt.show()

In [ ]:
clust_df = data.load_clust_df(clust_thresh=5)
clust_2013 = clust_df[clust_df.year == 2013]
clust_lens = map(lambda x: len(clust_2013[clust_2013.cluster == x]), clust_2013.cluster.unique())
over_300 = np.array(clust_lens)> 300
print "over 300 clusts: " + str(np.array(clust_2013.cluster.unique())[over_300])
plt.hist(clust_lens, bins=100)
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
#ak_bb = [lats[-1,0], lats[0,0], lons[0,0], lons[0,-1]]
mp = Basemap(projection="merc",
             lat_0=ak_bb[0], lon_0=ak_bb[0],
             llcrnrlat=ak_bb[0],
             llcrnrlon=ak_bb[2],
             urcrnrlat=ak_bb[1],
             urcrnrlon=ak_bb[3],
             resolution='i')

mp.drawcoastlines()
mp_lons, mp_lats = mp(np.array(clust.long), np.array(clust.lat))
mp.plot(mp_lons, mp_lats, 'r.', markersize=1)
plt.title("Number of detections in clust %d" % ct)
plt.savefig("pics/clust_%d.png" % ct, bbox_inches='tight')
plt.show()

In [ ]:
xs = []
ys = []
colors = []
min_day = np.min(clust.dayofyear)
max_day = np.max(clust.dayofyear)
for dayofyear in xrange(min_day, max_day+1):
    todays_fires = clust[clust.dayofyear==dayofyear]
    today_xs, today_ys = np.array(todays_fires.x), np.array(todays_fires.y)
    xs += list(today_xs)
    ys += list(today_ys)
    colors += [float(dayofyear)]*len(todays_fires) #/ (max_day - min_day)]*len(todays_fires)

xs = np.array(xs) - min(xs)
ys = np.array(ys) - min(ys)
matplotlib.rcParams.update({'font.size': 14})
plt.xlabel("km")
plt.ylabel("km")
plt.scatter(xs, ys, marker='o', s=30, c=colors, cmap=plt.get_cmap('YlOrRd'))
frame = plt.gca()
#frame.axes.get_xaxis().set_visible(False)
#frame.axes.get_yaxis().set_visible(False)
plt.colorbar()
#plt.savefig("pics/clust_%d_colormap.png" % ct, bbox_inches='tight')
#plt.show()
plt.close()

# Get position in GFS tensors of this location
days = np.arange(min_day, max_day+1)
fires_per_day = map(lambda x: len(clust[clust.dayofyear==x]), days)
lat_cent = np.mean(clust.lat)
lon_cent = np.mean(clust.long)
gfs_df = gfs_to_loc_df(gfs, lat_cent, lon_cent)
gfs_df = gfs_df[(gfs_df.year==2013) & (gfs_df.dayofyear <= max_day) & (gfs_df.dayofyear >= min_day)]

covars = ["n_det", "temp", "humidity", "wind", "rain"]
ticks = [[0,30,60,90,120], [294, 298, 302, 306, 310], [20, 30, 40, 50], [0,2,4,6], [0., .5, 1., 1.5, 2., 2.5]]
ylims = [[], (293, 310), (20, 55), (), ()]
titles = ["Active fire detection count y(t)", "Temperature (K)", "Humidity (%)", "Wind (m/s)", "Rain (mm/day)"]
fig, axes = plt.subplots(nrows=len(covars), ncols=1, figsize=(10,8))
for i, var in enumerate(covars):
    matplotlib.rcParams.update({'font.size': 15})
    if not i:
        ax = plt.subplot(511 + i)
        ax1 = ax
    else:
        ax = plt.subplot(511 + i, sharex=ax1)

    if var == 'n_det':
        plt.plot(days, fires_per_day, "r-", linewidth=3)
        plt.plot(days, fires_per_day, "rs", markersize=8)
    else:
        plt.plot(gfs_df.dayofyear, gfs_df[var], "r-", linewidth=3)
        plt.plot(gfs_df.dayofyear, gfs_df[var], "rs", markersize=8)
        
    if i != len(covars) - 1:
        plt.tick_params(axis='x', labelbottom='off') 
#        plt.setp(ax.get_xticklabels(), visible=False)
    else:
        plt.xlabel("Day of year (2013)")
    plt.yticks(ticks[i])
    if ylims[i]:
        plt.ylim(ylims[i])
    matplotlib.rcParams.update({'font.size': 13})
    plt.title(titles[i], fontweight='bold')
fig.tight_layout(h_pad=.1)
plt.savefig("pics/clust_%d_covars.png" % ct, bbox_inches='tight')

In [ ]:
lat_cent = np.mean(clust.lat)
lon_cent = np.mean(clust.long)
gfs_df = gfs_to_loc_df(gfs, lat_cent, lon_cent)
modis.iloc[0:5]

In [ ]:
ak_bb